This loads MGH LEADS data into redcap's MASTER project. Uses the RECON_FLAIR pipeline data.

In [366]:
import re
import getpass
import pandas as pd
import os
import pydicom
from pydicom.tag import Tag
import datetime
import glob
import math
import csv
from redcap import Project, RedcapError
import subprocess
import datetime
from ast import literal_eval
import numpy as np

api_url = 'https://redcap.partners.org/redcap/api/'
api_key = '49E6305CE4B41073E2342F723C0F93F8'
project = Project(api_url, api_key)

In [461]:
MGH_subjects = pd.read_csv('/autofs/cluster/animal/scan_data/leads/spreadsheets/IDENTIFICATION/MGH_SUBJECTS.csv')
MGH_subjects = MGH_subjects.replace(np.nan, '', regex=True)
# need to convert all columns with numbers to int --> str
RECON_FLAIR_NOTES = pd.read_csv('/autofs/cluster/animal/scan_data/leads/recon_nip/RECON_FLAIR/recon_notes.csv')
RECON_FLAIR_NOTES = RECON_FLAIR_NOTES.replace(np.nan, '', regex=True)


In [482]:
# put all subjects and add to the dtaframe so that all info is imported!
sublist = #['LSWA_F_58_20190417'] 

for sub in sublist:
    
    name = sub.split("_")[0]
    date = sub.split("_")[3]
    date_sep = date[0:4]+'-'+date[4:6]+'-'+date[6:10]
    
    #obtain leads ID
    subject_init = sub.split('_')
    subject = subject_init[0]+'_'+subject_init[1]+'_'+subject_init[2]
    data_frame = project.export_records(records=[subject],forms=['identification_numbers'],events=[('static_arm_1')],format='df')
    try:
        leads_id = 'LDS'+str(int(data_frame['leads_id'].values[0]))+'_'+date
    except(ValueError):
        leads_id = data_frame['leads_id'].values[0]+'_'+date
    print(leads_id)
    
    # find the redcap arm according to the date of the scan
    session_of_interest = project.export_records(records=[subject],forms=['scan_session'],format='df')
    session_new = session_of_interest.reset_index()
    find_arm = session_new[session_new['scan_date'] == date_sep]
    arm = find_arm.loc[: , "redcap_event_name"].values[0]
    print(arm)
    
    # export scan_session form content of specified arm
    session_edit = project.export_records(records=[subject],forms=['scan_session'],events=[(arm)],format='df')
    #session_edit = session_edit.reset_index()
    
    if MGH_subjects[MGH_subjects['dickerson_sessionid']==sub].XNAT_upload.values[0] == 0:
        scan_uploaded_to_xnat = '0'
        print('WARNING: scan not on xnat')
    else:
        scan_uploaded_to_xnat = '1'
    scan_session_series_notes = MGH_subjects[MGH_subjects['dickerson_sessionid']==sub].notes.values[0]
    
    cmd = [ 'findsession', name]
    output = subprocess.Popen( cmd, stdout=subprocess.PIPE ).communicate()[0]
    trials = [f for f in str(output).split("=======") if (("SUBJECT") in f)]

    for trial in trials:
        m = re.search('DATE   :  (.+?)TIME', trial)
        found = m.group(1).replace(",","").strip('\\n')
        date_time_obj = datetime.datetime.strptime(found, '%B %d %Y')
        date_redcapformat = str(date_time_obj.date())
        date_conv = date_redcapformat.replace("-","")
        if date_conv == date:
            dpath = trial.split(':')[-1][:-1].strip('\\n').replace(" ","")
            break
        else:
            pass
    
    # now set all variables that I will import
    scan_upload_path = dpath
    
    #yes / no
    scan_flag_for_review  = str(int((RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].FLAG_FOR_IR).values[0]))
    
    #notes
    scan_flag_trigger = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].IR_TRIGGER).values[0]

    # text (string)
    recon1_qc_path = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].RECON_PATH).values[0]

    # read in RECON_FLAIR notes
    # 5_3_0, v5.3 | 6_beta, v6 beta | 6_0_0, v6.0 | dev, dev
    FSV = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].FS_VERSION).values[0]
    if float(FSV) == 6:
        recon1_fsversion = '6_0_0'
    else:
        recon1_fsversion = ''

    # 1, good | 2, okay | 3, bad | 4, unknown | 5, flag
    recon_qc = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].Recon_Quality).values[0]
    if recon_qc == 'good':
        recon1_quality = '1'
    elif recon_qc == 'okay':
        recon1_quality = '2'
    elif recon_qc == 'bad':
        recon1_quality = '3'
    elif (recon_qc == 'unknown') or (recon_qc == ''):
        recon1_quality = '4'
    elif recon_qc == 'flag':
        recon1_quality = '5'
    
    # yes, yes | inprog, in progress | flag_needs_xopts, flag - needs troubleshooting/xopts | no, no | exclude, exclude from processing (ie. no highres T1)
    recon_status = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].QC_STATUS).values[0]
    if recon_status == 'complete':
        recon1_qc_status = 'yes'
    elif recon_status == 'error':
        recon1_qc_status = 'flag_needs_xopts'
    else:
        recon1_qc_status = 'inprog'
    
    recon1_username = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].EDITOR).values[0]

    recon1_qc_notes = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].FS_QC_Notes).values[0]

    # now use the pandas data frame but replace the value of interest ; then try importing.
    
    session_edit.loc[:,'scan_uploaded_to_xnat'] = scan_uploaded_to_xnat
    #####session_edit.loc[session_edit['subject_id']==subject,'scan_session_series_notes'] = scan_session_series_notes
    session_edit.loc[:,'scan_upload_path'] = scan_upload_path
    session_edit.loc[:,'scan_flag_for_review'] = scan_flag_for_review
    session_edit.loc[:,'scan_flag_trigger'] = scan_flag_trigger
    session_edit.loc[:,'recon1_qc_path'] = recon1_qc_path
    session_edit.loc[:,'recon1_fsversion'] = recon1_fsversion
    session_edit.loc[:,'recon1_quality'] = recon1_quality
    session_edit.loc[:,'recon1_qc_status'] = recon1_qc_status
    session_edit.loc[:,'recon1_username'] = recon1_username
    session_edit.loc[:,'recon1_qc_notes'] = recon1_qc_notes
    session_edit.loc[:,'scan_uploaded_to_xnat'] = scan_uploaded_to_xnat
    #import!
    response = project.import_records(session_edit, date_format='YMD',format='df')

LDS3600087_20190417
static_arm_1
